#«Разработка автоматической диалоговой системы на основе языковой модели для анализа экономических новостей»



##Загрузка необходимых библиотек

In [3]:
!pip install langchain_openai==0.3.0 -q
!pip install langchain_community==0.3.14 -q
!pip install chromadb==0.5.0 -q
!pip install sentence_transformers==3.3.1 -q
!pip install langchain_huggingface==0.1.2 -q
!pip install langchain_chroma==0.2.0 -q
!pip install langchain_core==0.3.29 -q
!pip install pydantic==2.10.5 -q
!pip install onnxruntime==1.20.1 -q
!pip install feedparser -q
!pip install requests -q
!pip install telebot -q
!pip install gradio -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-text-splitters 0.3.8 requires langchain-core<1.0.0,>=0.3.51, but you have langchain-core 0.3.29 which is incompatible.
langchain 0.3.25 requires langchain-core<1.0.0,>=0.3.58, but you have langchain-core 0.3.29 which is incompatible.


In [1]:
import requests
import os
from bs4 import BeautifulSoup
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from google.colab import userdata
from langchain.schema.output_parser import StrOutputParser
from langchain_huggingface import ChatHuggingFace
from transformers import pipeline
import telebot

##Парсинг новостей

Данные, на основе которых модель будет отвечать на вопросы пользователя будут взяты с сайта 'https://www.rbc.ru/economics/'

Для парсинга воспользуемся библиотеками BeautifulSoup и requests

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
main_url = 'https://www.rbc.ru/economics/'

response = requests.get(main_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
#находим все ссылки на интересующие нас статьи(нас интересуют те, которые попадают в рубрику "экономика")
links = []
for tag in soup.find_all('a', href=True):
    href = tag['href']
    if href.startswith('https://www.rbc.ru/') and '/economics/' in href:
        links.append(href)

#оставляем только уникальные ссылки
links = list(set(links))
print(links)

['https://www.rbc.ru/economics/20/05/2025/682c42a39a794735ffe95a55', 'https://www.rbc.ru/economics/22/05/2025/682e80d09a794773e6b1fa6e', 'https://www.rbc.ru/economics/21/05/2025/682de33c9a794756f79632ef', 'https://www.rbc.ru/economics/27/05/2025/68355e739a7947343fa68c4a', 'https://www.rbc.ru/economics/20/05/2025/682b930a9a7947dc8c10ff8e', 'https://www.rbc.ru/economics/20/05/2025/682c22bc9a7947733044a0cf', 'https://www.rbc.ru/economics/21/05/2025/682ce0589a7947c3ec5cd612', 'https://www.rbc.ru/economics/27/05/2025/6835a5c79a79477cbf2ba6b9', 'https://www.rbc.ru/economics/?utm_source=topline', 'https://www.rbc.ru/economics/', 'https://www.rbc.ru/economics/27/05/2025/683568ed9a79472dae277da8', 'https://www.rbc.ru/economics/23/05/2025/682fdcf69a794703803b86bd', 'https://www.rbc.ru/interview/economics/21/05/2025/6829a8669a79473f7917e4e3', 'https://www.rbc.ru/economics/20/05/2025/682b9cd39a79471d6050da25', 'https://www.rbc.ru/economics/27/05/2025/6835bf899a7947ce81b0f3ed', 'https://www.rbc.ru/

In [ ]:
def get_article_text(url): #пишем функцию, которая будет извлекать текст статьи
    try:
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, 'html.parser')

        content = soup.find('div', {'class': 'article__text'}) #пробуем найти текст статьи внутри этих тегов
        if not content:
            content = soup.find('div', {'class': 'l-container'})  #если первый вариант считывания не сработал

        paragraphs = content.find_all('p')
        text = '\n'.join(p.get_text(separator=" ", strip=True) for p in paragraphs)

        return (url, text)

    except Exception as e:
        pass

In [ ]:
#получаем список ссылок и текстов всех новостей
texts_n_links = []
for link in links:
    article_text = get_article_text(link)
    texts_n_links.append(article_text)

texts_n_links

[('https://www.rbc.ru/economics/20/05/2025/682c42a39a794735ffe95a55',
  'Центробанк использует возможности искусственного интеллекта (ИИ), но он не может принимать решения вместо экспертов, в частности, по ключевой ставке, заявила председатель Банка России Эльвира Набиуллина на «Альфа-Саммите».\n«Можно\xa0ли нас заменить искусственным интеллектом при принятии решения о ключевой, хотя\xa0бы чуть-чуть? Сначала об искусственном интеллекте: мы действительно стараемся использовать возможности искусственного интеллекта. Я лично считаю, что там есть большой потенциал, но пока не в макроэкономическом анализе и прогнозировании»,\xa0— отметила глава ЦБ.\nНабиуллина пояснила, что ИИ учится на больших рядах данных, взятых из прошлого. Сейчас «по историческим меркам\xa0у нас не очень большие ряды данных», указала председатель Банка России. Кроме того, в условиях «структурной трансформации, когда старые связи между данными не работают, вообще выводы моделей, не только искусственного интеллекта, они 

##Получение эмбеддингов новостей

Так как далее полученные тексты новостей мы будем передавать в LLM, то никакую специальную предобработку текстов производить не будем и ограничимся лишь получением эмбеддингов новостей

In [ ]:
#получаем список только текстов новостей и преобразовываем их в Document
news_list = [elem[1] for elem in texts_n_links if elem]
documents = [Document(page_content=news) for news in news_list]

In [ ]:
#загружаем сплиттер
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100)

In [ ]:
split_docs = splitter.split_documents(documents)
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2" #выбираем модель - энкодер новостей
hf = HuggingFaceEmbeddings(
    model_name=model_name,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#в качестве базы данных для хранения эмбеддингов новостей выбрали Chroma
db = Chroma.from_documents(
    split_docs,
    hf,
    persist_directory="tmp"
)

In [ ]:
#модель будет давать пользователю ответ на основе топ-2 результатов, которые вернет retriever
retriever = db.as_retriever(search_kwargs={"k": 2})

## Определение моделей для ответа

Следующий список моделей будет использоваться для ответа на вопрос пользователя.

Если у первой модели будет превышен лимит запросов к ней или будет выявлена какая-либо ошибка при генерации ответа, то будет использована следующая модель из списка.

In [ ]:
models = [
    'mistralai/mistral-7b-instruct:free',
    "meta-llama/llama-4-maverick:free",
    "deepseek/deepseek-chat-v3-0324:free",
    "mistralai/mistral-small-3.1-24b-instruct:free",
]


Следующая функция будет определять, связан ли вопрос заданный пользователем с новостями или он общается на абстрактные темы. Это позволит повысить точность ответов модели, а также без необходимости не извлекать документы из базы данных.

В зависимости от результата этой проверки будет использоваться либо модель, отвечающая на вопрос на основе новостей, либо модель, которая просто умеет вежливо поддерживать диалог.

In [ ]:
def check_is_about_news(question):
    system_prompt_classifier = SystemMessagePromptTemplate.from_template('''Ты — классификатор. Твоя задача — определить, хочет ли пользователь получить информацию о текущих новостях, событиях или свежих фактах из мира.

    Если сообщение явно или косвенно спрашивает о новостях, ответь только "да".
    Если это приветствие, благодарность, комплимент или обычный разговор — ответь только "нет".

    Примеры:
    "что сегодня пишут про японию?" -> да
    "привет, спасибо, ты хороший бот" -> нет
    "какая погода в Москве?" -> нет
    "расскажи анекдот" -> нет
    "есть что-то про трампа?" -> нет. Ничего кроме "да" и "нет" писать нельзя!''')
    human_prompt_classifier = HumanMessagePromptTemplate.from_template('Пользователь задал вопрос: {question}. Верни "да", если пользователь спрашивает про новости и "нет" в инои случае.')
    chat_prompt_classifier = ChatPromptTemplate.from_messages([system_prompt_classifier, human_prompt_classifier])

    for model_name in models:
        try:
            model = ChatOpenAI(
                base_url="https://openrouter.ai/api/v1",
                api_key=userdata.get('firstkey'),
                model_name=model_name,
            )
            chain = chat_prompt_classifier | model | StrOutputParser()
            return chain.invoke({"question": question})

        except Exception as e:
            continue
    return "Не удалось получить ответ: все модели недоступны."

Следующая функция будет использована, если check_is_about_news вернет положительный ответ(то есть вопрос, заданный пользователем, связан с экономическими новостями)

In [ ]:
def run_with_docs(question, docs):
    system_prompt = SystemMessagePromptTemplate.from_template('Ты — помощник, отвечающий на вопросы на основе предоставленных тебе новостей.')
    human_prompt = HumanMessagePromptTemplate.from_template('Ответь на вопрос: {question}. Вот несколько новостей, которые могут помочь ответить на этот вопрос: {docs}. Дай ответ, основанный только на предоставленных новостя. Если ты не уверен в ответе, то вовпроси уточнить вопрос или напиши "я не уверен, что могу ответить на этот вопрос"')
    chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
    for model_name in models:
        try:
            model = ChatOpenAI(
                base_url="https://openrouter.ai/api/v1",
                api_key=userdata.get('firstkey'),
                model_name=model_name,
            )
            chain = chat_prompt | model | StrOutputParser()
            return chain.invoke({"question": question, "docs": docs})
        except Exception as e:
            continue
    return "Не удалось получить ответ: все модели недоступны."

Следующая функция будет использована, если check_is_about_news вернет отрицательный ответ(то есть вопрос, заданный пользователем, никак не связан с экономическими новостями)

In [ ]:
def run_without_docs(question):
    system_prompt = SystemMessagePromptTemplate.from_template('Ты — ассистент пользователя, который с ним мило болтает и вежливо отвечает на его сообщения.')
    human_prompt = HumanMessagePromptTemplate.from_template('Пользователь задал вопрос: {question}. Вежливо ответь на его сообщение.')
    chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
    for model_name in models:
        try:
            model = ChatOpenAI(
                base_url="https://openrouter.ai/api/v1",
                api_key=userdata.get('firstkey'),
                model_name=model_name,
            )
            chain = chat_prompt | model | StrOutputParser()
            return chain.invoke({"question": question})
        except Exception as e:
            continue
    return "Не удалось получить ответ: все модели недоступны."

Функция answer_question возвращает финальный ответ на вопрос пользователя

In [ ]:
def answer_question(question):
    check = check_is_about_news(question)
    if 'да' in check.lower():
      docs = [elem.page_content for elem in retriever.invoke(question)]
      response = run_with_docs(question, docs)

    elif 'нет' in check.lower():
      response = run_without_docs(question)

    else:
      response = "Не удалось получить ответ: все модели недоступны."
    return response

##Реализация телеграм-бота

@EconomistAssistant_bot

In [ ]:
bot = telebot.TeleBot(userdata.get('tg_token'));

@bot.message_handler(content_types=['text'])
def get_text_messages(message):
  if message.text.lower() == "Привет":
    bot.send_message(message.from_user.id, "Привет, чем я могу тебе помочь?")

  elif message.text == "/help":
      bot.send_message(message.from_user.id, 'Напиши "Привет"')

  else:
      bot.send_message(message.from_user.id, answer_question(message.text))

bot.polling(none_stop=True, interval=0)

<ipython-input-13-9cb2363d996b>:18: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(


##Реализация диалогового интерфейса с помощью gradio

(запасной вариант)

In [ ]:
import gradio as gr

gr.Interface(
    fn= answer_question,
    inputs=gr.Textbox(label="Введите вопрос"),
    outputs=gr.Textbox(label="Ответ ассистента"),
    title="Ваш ассистент по экономическим новостям",
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://007b12d2d1e2ccf151.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
